# Matrix Calculus (18.063) Pset 1 Solutions

This notebook contains computational solutions for problem set 1 of *18.063: Matrix Calculus* at MIT in IAP 2025.  See also the solutions PDF for mathematical derivations and analytical results.

## Problem 2

We are asked to check that $\nabla_A{\lambda} = v v^T$, where $\lambda$ is some eigenvalue of a real-symmetric matrix $A$ and $v$ is the corresponding unit-normalized eigenvector, using a random $4 \times 4$ matrices.   

We can generate a random real-symmetric matrix by $A = B + B^T$ where $B$ is an arbitrary real matrix:

In [1]:
B = randn(4,4)
A = B + B'

4×4 Matrix{Float64}:
 -0.482936   -0.715913  -1.70091   -0.0236887
 -0.715913   -3.56332   -0.732205  -1.47105
 -1.70091    -0.732205   0.253464   0.182552
 -0.0236887  -1.47105    0.182552  -2.75639

Let's also create a random small perturbation.  It turns out that the formula works even if the perturbation is not real-symmetric, in fact:

In [2]:
δA = randn(4,4) * 1e-8

4×4 Matrix{Float64}:
  1.02237e-11   2.32632e-8   2.32533e-8   2.20845e-8
 -1.03699e-8   -1.88636e-9   1.04404e-8  -3.11713e-9
 -9.00341e-9   -9.42277e-9   8.60412e-9   9.62347e-9
 -3.92094e-9    1.24847e-8  -1.64921e-9   1.21897e-8

It was sufficient to just check one eigenvalue, say the smallest one, but let's just check all of them.

The `eigen(A)` function in the LinearAlgebra library will give us the eigenvalues and eigenvectors, and it automatically sorts the eigenvalues and normalizes the eigenvectors to unit magnitude so we don't have to worry about that:

In [3]:
using LinearAlgebra

λ, V = eigen(A)
δλ = eigvals(A + δA) - λ

4-element Vector{Float64}:
  1.188114673311702e-8
  2.027832124440465e-9
  6.5526576387142654e-9
 -1.5439416412021956e-9

Now, let's try the formula, applied to each of the 4 eigenvectors.  Note that the columns of $V$ are the eigenvectors, so `λ[i]` corresponds to `V[:,i]`.  Also, note that `dot` in Julia gives the Frobenius inner product for two matrices.

In [4]:
dλ = [ dot(V[:,i] * V[:,i]', δA) for i = 1:4 ]

4-element Vector{Float64}:
  1.1881160500284657e-8
  2.0278345838592953e-9
  6.552662002898275e-9
 -1.5439416493947632e-9

Beautiful, they all match $\delta \lambda$ to a few significant digits (about as much as we can expect from a simple finite-difference approximation like this).

You can run the whole thing a few times if you want:

In [5]:
for _ = 1:10
    B = randn(4,4)
    A = B + B'
    δA = randn(4,4) * 1e-8
    λ, V = eigen(A)
    δλ = eigvals(A + δA) - λ
    dλ = [ dot(V[:,i] * V[:,i]', δA) for i = 1:4 ]
    @show norm(δλ - dλ) / norm(dλ) # relative error
end

norm(δλ - dλ) / norm(dλ) = 1.3998932906429625e-6
norm(δλ - dλ) / norm(dλ) = 6.384624638425199e-7
norm(δλ - dλ) / norm(dλ) = 1.1994572440833222e-6
norm(δλ - dλ) / norm(dλ) = 2.032401443943924e-6
norm(δλ - dλ) / norm(dλ) = 6.35272235832111e-7
norm(δλ - dλ) / norm(dλ) = 4.1804100802261764e-7
norm(δλ - dλ) / norm(dλ) = 7.372242201918826e-7
norm(δλ - dλ) / norm(dλ) = 9.663838924444233e-7
norm(δλ - dλ) / norm(dλ) = 6.961791149971516e-7
norm(δλ - dλ) / norm(dλ) = 2.535789068234015e-7


and we find that the relative error $\Vert d\lambda - \delta\lambda \Vert / \Vert d\lambda \Vert$ is always small.

## Problem 3 (optional numerical checks)

You were *not* required to computationally validate your solutions on this problem, but it is always a good idea to do a quick finite-difference check, especially when the answer involved a bunch of algebra where you could easily make a mistake.

We pick a random $x \in \mathbb{R}^5$ and $\delta x \in \mathbb{R}^5$ on the order of $10^{-8}$ for use in all examples, and/or $A \in \mathbb{R}^{5 \times 5}$ and $\delta A \in \mathbb{R}^{5 \times 5}$, and a function `relerr` to compute relative errors, which should be $\ll 1$ compared to finite differences if our answers are correct.

In [6]:
using LinearAlgebra # for norm
relerr(approx, exact) = norm(approx - exact) / norm(exact)

x = randn(5)
δx = randn(5) * 1e-8
A = randn(5,5)
δA = randn(5,5) * 1e-8;

### 3.1

$f(x) = g.(x)$ gives $f' = \mathrm{Diagonal}(g'.(x))$.  Check for $g(x) = \sin(x)$:

In [7]:
f_1(x) = sin.(x)
f_1′(x) = Diagonal(cos.(x))
relerr(f_1(x + δx) - f_1(x), f_1′(x) * δx)

6.345900691824831e-9

### 3.2

$f(x) = (A^T A)^{-1}$ gives
$f'(A)[dA] = -f(A) (dA^T \, A + A^T \, dA) f(A)$
Check:

In [8]:
f_2(A) = (A'A)^-1
f_2′(A) = dA -> -f_2(A) * (dA' * A + A' * dA) * f_2(A)
relerr(f_2(A + δA) - f_2(A), f_2′(A)(δA))

6.479231577342702e-8

### 3.3

$f(x) = (I + xx^T)^{-1} x$ gives 
$$
f'(x) = (1 - x^T f(x))  (I + xx^T)^{-1} - f(x) f(x)^T
$$
Check:

In [9]:
f_3(x) = (I + x*x') \ x
f_3′(x) = let f = f_3(x); (1 - x'f) * (I + x*x')^-1 - f*f'; end
relerr(f_3(x + δx) - f_3(x), f_3′(x) * δx)

3.3268069443429275e-8

### 3.4

$f(A) = \mathrm{trace}(A^3)$ gives $\nabla f = (3A^2)^T$.  Check:

In [10]:
f_4(A) = tr(A^3)
∇f_4(A) = (3A^2)'
relerr(f_4(A + δA) - f_4(A), ∇f_4(A) ⋅ δA)

1.795662972343922e-9

(Note that `A ⋅ B` is equivalent to `dot(A, B)` in Julia, and for matrices this is the Frobenius inner product.)

**Hooray**, it all worked!

## Problem 5.3

In part 2, we showed that a Newton step for $f(\lambda) = \det(M(\lambda))$ is 
$$
\lambda \to \lambda - f'(\lambda)^{-1} f(\lambda) = \lambda - \mathrm{trace}[M(\lambda)^{-1} M'(\lambda)]^{-1} \, .
$$

Now, we are going to check it for $M(\lambda) = A - \lambda I + \alpha \lambda \sin(\lambda) B$ for $\alpha = 0.01$ and given matrices $A, B$, using as a starting guess $\lambda_0$ the largest eigenvalue of $A$.

Note that the derivative of $M$ is simply $M'(\lambda) = \alpha (\sin(\lambda) + \lambda \cos(\lambda)) B - I$.

In [11]:
A = [-2 -1 -7; -1 6 5; -7 5 6]
B = [7 -1 8; -1 7 -1; 8 -1 3]
α = 1//100 # exact rational coefficient

M₅(λ) = A - λ*I + α*λ*sin(λ)*B
M₅′(λ) = α * (sin(λ) + λ*cos(λ)) * B - I

λ₀ = eigvals(A)[end] # largest eigenvalue of A as starting guess

13.310708435174291

Now, let's do 5 Newton steps, printing out the change $\Delta\lambda$ on each step so we can watch it converge:

In [12]:
λ = λ₀
for _ = 1:5
    @show Δλ = -1 / tr(M₅(λ) \ M₅′(λ))
    λ += Δλ
end
@show λ

Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = 0.013878454139803088
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = -6.179777913448486e-6
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = -1.2943160456183902e-12
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = 4.851259541298374e-16
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = 4.851259541298374e-16
λ = 13.324580709534885


13.324580709534885

As is typical for Newton's method, the error roughly *squares* on each step.  Since our initial guess was correct to $\approx 0.01$ (thanks to the fact that $\alpha$ is small so that it is nearly a linear eigenproblem), it converges to machine precision in only 3 steps!

The final answer of $\boxed{\lambda \approx 13.324580709534885}$ should be accurate to more than 15 significant digits. 

In fact, if we want, we could get *many more* digits by switching to arbitrary-precision arithmetic, using Julia's `BigFloat` type, which defaults to about 77 significant digits.

In `BigFloat` arithmetic, we have to be a bit careful that all of our parameters are entered exactly.  That's why we entered $\alpha$ as `1//100` above, which is Julia syntax for the exact rational $\frac{1}{100}$, rather than as the floating-point value `0.01` (which is actually rounded to a slightly nearby value).  Our initial guess can still be the same, however, since that is approximate anyway.

Thanks to the rapid ("quadratic") convergence, it only takes 5–6 steps to converge to 77 digits:

In [13]:
λ = BigFloat(λ₀)
for _ = 1:6
    @show Δλ = -1 / tr(M₅(λ) \ M₅′(λ))
    λ += Δλ
end
@show λ

Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = 0.01387845413980278146924381478102036939592172873690338958398336900711028713973202
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = -6.179777913283799906559965371941459950162437546405999559750023965330840318196745e-06
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = -1.29483782619046423801795945492132904414631967410863748091816343685228771211928e-12
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = -5.684469209316674137521208519556973673669745894000627686884323646805251349135944e-26
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = -1.095567143853168221946500022463955277722520254619605286171673829733615056171145e-52
Δλ = -1 / tr(M₅(λ) \ M₅′(λ)) = -3.773682780296177136654839871911331258140476718388645796628924966406292865188501e-77
λ = 13.32458070953488591297670726457428893073423073704312535878386385948794565820632


13.32458070953488591297670726457428893073423073704312535878386385948794565820632

So, an even more exact answer, to 73 decimal places, is $\lambda \approx 13.3245807095348859129767072645742889307342307370431253587838638594879456582$.

Comparing to our double-precision answer above, we see that we got 16 significant digits correct.

## Problem 6.1

Let's check the formula for the derivative of a matrix exponential with a random $3\times 3$ matrix $A$ and a small perturbation $\delta A$:

In [14]:
using LinearAlgebra
A = randn(3,3)

3×3 Matrix{Float64}:
 -0.425224  2.18766     0.518162
  1.32677   0.0196914   0.00593049
 -1.58095   1.28556    -0.891346

In [15]:
δA = randn(3,3) * 1e-8

3×3 Matrix{Float64}:
  2.00187e-9   9.20261e-9  -8.75115e-9
  1.11933e-9  -3.24089e-9  -9.03447e-10
 -1.31247e-9   1.16987e-8   8.60381e-9

First, we'll compute the finite difference:

In [16]:
approx = exp(A + δA) - exp(A)

3×3 Matrix{Float64}:
  1.36821e-8  1.62146e-8  -4.71975e-9
  5.72124e-9  4.1576e-9   -4.31738e-9
 -1.52379e-9  5.60998e-9   5.33695e-9

Now, we'll use the analytical expression via $M = \begin{pmatrix} A & \delta A \\ & A \end{pmatrix}$:

In [17]:
M = [ A  δA
     0I   A ]
exact = exp(M)[1:3, 4:6]

3×3 Matrix{Float64}:
  1.36821e-8  1.62146e-8  -4.71975e-9
  5.72124e-9  4.1576e-9   -4.31738e-9
 -1.52379e-9  5.60998e-9   5.33695e-9

By inspection, they match pretty well!  Let's be quantitative using the `relerr` function we defined for problem 3:

In [18]:
relerr(approx, exact)

6.034023320947324e-8

They match to almost 8 digits, which is limited by the accuracy of the finite-difference approximation.  Success!